# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Depression"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [1]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_Lower_Grade_Glioma_(LGG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_LGG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LGG.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'additional_surgery_metastatic_procedure',
 'age_at_initial_pathologic_diagnosis']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', '

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [31.0, 67.0, 44.0, 37.0, 50.0],
 'days_to_birth': [-11509.0, -24578.0, -16297.0, -13565.0, -18494.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, 234.0, 1335.0, 1106.0, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [3574.0, nan, nan, 481.0, 1828.0],
 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_of_animal_insect_allergy': ['12 - 20 Years',
  nan,
  nan,
  nan,
  nan],
 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2005.0,
  2005.0,
  2006.0,
  2009.0,
  2010.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lower Grade Glioma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-CS-4938-01,1,31.0,0.0,1.675408,2.256574,-0.025535,-0.129672,0.512522,1.00569,0.980606,...,2.145814,-0.086682,-0.748878,-0.411617,0.508467,-1.28139,0.587923,-1.931327,-1.200995,-5.525533
TCGA-CS-4941-01,1,67.0,1.0,1.677108,1.531874,-0.103335,-0.044972,1.044022,0.68629,0.715906,...,3.563814,-0.086682,2.294922,2.043683,0.703167,-1.28139,-0.211777,-0.691827,-0.067395,-4.274433
TCGA-CS-4942-01,1,44.0,0.0,1.451408,0.339574,0.227665,-0.359972,0.479422,0.83969,0.928806,...,2.061014,-0.086682,-0.748878,2.006583,1.287867,-1.28139,-0.521377,-1.996927,-1.020895,-5.096833
TCGA-CS-4943-01,1,37.0,1.0,1.159508,1.747574,0.193265,0.049128,0.345022,0.20049,0.736706,...,-0.864186,-0.086682,-0.748878,-0.862817,0.231767,-1.28139,0.760023,-1.260727,-2.100995,-4.541333
TCGA-CS-4944-01,1,50.0,1.0,0.847608,3.430774,-0.531035,-0.503272,0.820222,1.21089,0.934706,...,3.457214,-0.086682,-0.159678,1.123883,0.491667,-1.28139,-0.713077,-1.252627,-1.109495,-3.516133


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 529 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lower Grade Glioma', the least common label is '1' with 529 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lower Grade Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 32.0
  50% (Median): 41.0
  75%: 53.0
Min: 14.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 238 occurrences. This represents 44.99% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Depression"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE222468', 'GSE242736', 'GSE201332', 'GSE208668']

Repeat the below steps for all the accession numbers

In [5]:
cohort = accession_num = "GSE201332"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Depression/GSE201332/GSE201332_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Depression/GSE201332/GSE201332_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [34]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Peripheral whole blood: Healthy control and Major depression disorder patients"
!Series_summary	"Transcriptional profiling in the whole blood samples of healthy controls and major depression disorder (MDD) patients who have never been treated with depression medication. Samples included 20 healthy controls and 20 MDD patients who have never been treated with depression medication. Goal was to discover the differentially expressed genes."
!Series_overall_design	"20 Healthy controls vs. 20 major depression disorder (MDD) patients"


In [7]:
clinical_data.head()

,!Sample_geo_accession,GSM6058641,GSM6058642,GSM6058643,GSM6058644,GSM6058645,GSM6058646,GSM6058647,GSM6058648,GSM6058649,...,GSM6058671,GSM6058672,GSM6058673,GSM6058674,GSM6058675,GSM6058676,GSM6058677,GSM6058678,GSM6058679,GSM6058680
0,!Sample_characteristics_ch1,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,...,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood
1,!Sample_characteristics_ch1,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,subject status: heathy controls,...,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients,subject status: MDD patients
2,!Sample_characteristics_ch1,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,...,gender: male,gender: male,gender: male,gender: female,gender: female,gender: female,gender: female,gender: female,gender: male,gender: male
3,!Sample_characteristics_ch1,age: 48y,age: 33y,age: 43y,age: 24y,age: 24y,age: 45y,age: 36y,age: 59y,age: 51y,...,age: 47y,age: 25y,age: 54y,age: 47y,age: 25y,age: 28y,age: 52y,age: 33y,age: 30y,age: 51y


In [8]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: whole blood'],
 1: ['subject status: heathy controls', 'subject status: MDD patients'],
 2: ['gender: male', 'gender: female'],
 3: ['age: 48y',
  'age: 33y',
  'age: 43y',
  'age: 24y',
  'age: 45y',
  'age: 36y',
  'age: 59y',
  'age: 51y',
  'age: 26y',
  'age: 25y',
  'age: 32y',
  'age: 39y',
  'age: 41y',
  'age: 52y',
  'age: 53y',
  'age: 44y',
  'age: 22y',
  'age: 47y',
  'age: 54y',
  'age: 28y',
  'age: 30y']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [9]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Depression\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Depression\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an inte

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [32]:
is_gene_availabe = False
trait_row_id = 1
age_row_id = 3
gender_row_id = 2

trait_type = 'binary'

In [33]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [12]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    if value == 'subject status: heathy controls':
        return '0'
    elif value == 'subject status: MDD patients':
        return '1'
    else:
        return None



def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format



def convert_gender(value):
    # For 'gender': Binary
    if value == 'gender: female':
        return '1'
    elif value == 'gender: male':
        return '0'
    else:
        return None  # Unknown gender



In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [23]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

In [25]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'SPOT_ID'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [27]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [30]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [31]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [ ]:
ranked_df.head()

In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [ ]:
cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

The cross-validation accuracy is 74.48% ± 3.52%


In [ ]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)